In [14]:
import os
import time
import zipfile
import pickle
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim, SparseRandomProjection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Random Forest with GridSearchCV and Multiple Scoring Metrics

def flastRandomForestWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, dim, eps):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    if len(dataPoints) > 0:
        print(f"Sample document: {dataPoints[0]}")

    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization
    Z = flastVectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = np.array([Z[i].toarray() for i in range(Z.shape[0])])
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define the pipeline for Random Forest
    pipeline = Pipeline([
        ('rf', RandomForestClassifier(random_state=42))
    ])

    # Define parameter grid for hyperparameter tuning
    '''
    param_grid = {
        'rf__n_estimators': [100, 200, 300],  # Number of trees in the forest
        'rf__max_depth': [10, 20, 30],  # Maximum depth of the tree
        'rf__criterion': ['gini', 'entropy'],  # Criteria for splitting
    }
    
    '''
    
    param_grid = {
            "rf__n_estimators": [50, 100, 200],
            "rf__criterion": ["gini", "entropy"],
            "rf__max_depth": [10, 30, 50, 100, 300, 500],
            "rf__min_samples_split": [2, 5],
            "rf__min_samples_leaf": [1, 2]
        }

    # Custom scoring functions for precision, recall, and accuracy
    scoring = {
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score)
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='accuracy', verbose=1, return_train_score=True)

    # Fit the GridSearchCV on training data
    grid_search.fit(dataPointsList.reshape(len(dataPointsList), -1), dataLabelsList)

    # Get the best parameters and the best score for accuracy
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy Score: {best_score}")

    # Save the results
    outFile = f"params-rf-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("n_estimators,max_depth,criterion,accuracy,precision,recall,f1,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_recall'][idx]
            preparationTime = vecTime / len(dataPointsList)  # Estimating preparation time
            fo.write(f"{param['rf__n_estimators']},{param['rf__max_depth']},{param['rf__criterion']},{accuracy},{precision},{recall},{f1},{preparationTime}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "cleaned_flaky_files.zip"
    nonFlakyZip = "reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results_RandomForst"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    # Perform Random Forest analysis for 5 folds and 3 folds
    dim = 100  # Example value for JL dimensionality reduction
    eps = 0.3  # JL epsilon

    print("Starting 5-fold analysis...")
    best_params_5folds, best_score_5folds = flastRandomForestWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 5, dim, eps)

    print("Starting 3-fold analysis...")
    best_params_3folds, best_score_3folds = flastRandomForestWithGridSearchCV(outDir, flakyZip, nonFlakyZip, extractDir, 3, dim, eps)

    print("Best results for 5-fold:")
    print(f"Best Parameters: {best_params_5folds}")
    print(f"Best Accuracy Score: {best_score_5folds}")

    print("Best results for 3-fold:")
    print(f"Best Parameters: {best_params_3folds}")
    print(f"Best Accuracy Score: {best_score_3folds}")
    
    
    
    
    '''
    
    take best parameters
    fit them inside model
    hypertune
    visualize use heatmap
    overleaf(writing)
    

    
    '''


Starting 5-fold analysis...
Number of flaky documents: 47
Number of non-flaky documents: 47
Total number of documents: 94
Sample document: # -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2020.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""General utility functions for testing."""

from qiskit import QuantumCircuit
from qiskit.providers.ibmq.ibmqbackend import IBMQBackend


def most_busy_backend(provider):
    """Return the most busy backend for the provider given.

    Return the most busy available backend for those that
    have a `pending_jobs` in their `status`. Backends such as
    